# Winter School on Tensor Network Methods for Quantum Many-Body Systems: Simple ITEBD Algorithm 

In [1]:
from ncon import ncon
import numpy as np
import scipy.linalg as scl
from scipy import integrate

## iTEBD Simulation of 1D Ising Model In Transverse Field (ITF)

$$H=-J\sum_{ij} \sigma_i^z \sigma_j^z-h\sum_i \sigma_i^x$$

In [9]:
#--- Pauli Operators ----
SigmaX=np.array([[0,1],
                 [1,0]])

SigmaZ=np.array([[1,0],
                 [0,-1]])

Identity=np.array([[1,0],
                   [0,1]])

#####    Problem Parameters   ######
#--- Ising ferromagnetic exchange coupling --- 
J=-1

#--- Magnetic field strength --- 
h=0.2

#--- Imaginary-time evolution delta: delta=beta/n ---
delta=0.01

#--- Imaginary-time evolution Iteration ---
niter=4000

#--- Virtual Bond dimension ---
D=10

#--- Physical Bond dimension: For Pauli Spins p=2 for {up, down}  ---
p=2

### Building Hamiltonian Two-Body Gates
<img src="images/zzgate.png" width="350"> 

In [10]:
#--- Ising gate ---  
Hzz=ncon([SigmaZ,SigmaZ],[[-1,-3],[-2,-4]])

#--- Magnetic field gate (we build field one-site gate as a sum of two-body gates) ---
Hf=0.5*( ncon([SigmaX,Identity],[[-1,-3],[-2,-4]]) + ncon([Identity,SigmaX],[[-1,-3],[-2,-4]]) )

#--- Ising Hamiltonian --- 
H_ITF=J*Hzz-h*Hf
print("Shape H_ITF:",np.shape(H_ITF))

#--- ITE Gate ---
Hexpm=np.reshape(H_ITF,[p*p,p*p])
Ug=scl.expm(-delta*Hexpm)
Ug=np.reshape(Ug,[p,p,p,p])

Shape H_ITF: (2, 2, 2, 2)


### Initializing Tensors
<img src="images/unitcell.png" width="300"> 

In [11]:
#--- Gamma Tensors --- 
GA=np.random.rand(D,D,p); GB=np.random.rand(D,D,p)

#--- Normalizing Gamma Tensors so that their elements remain <1 ---
GA=GA/np.max(np.abs(GA));  GB=GB/np.max(np.abs(GB));
Gamma=[GA,GB]

#--- Lambda Matrices (Since they are diagonal, we only keep diagonal elements as 1D array) --- 
LA=np.random.rand(D); LB=np.random.rand(D)

#--- Normalizing Lambda Matrices so that: tr(LA)=1, tr(LB)=1 --- 
LA=LA/np.trace(np.diag(LA));  LB=LB/np.trace(np.diag(LB))

Lambda=[LA,LB]

### Main ITE Loop: Updating theta
<img src="images/odd_even_bonds.png" width="600"> 
<img src="images/ite_update.png" width="800">
<img src="images/theta.png" width="200"> 

In [12]:
#----Main ITE Iterations-----
for steps in range(niter):
    if steps%2==0:
        A=0; B=1
    else:
        A=1; B=0
        
    # theta tensor
    tensors=[np.diag(Lambda[B]),Gamma[A],np.diag(Lambda[A]),Gamma[B],np.diag(Lambda[B]),Ug]
    connect=[[-1,1],[1,2,3],[2,4],[4,5,6],[5,-3],[3,6,-2,-4]]
    order=[1,4,5,2,3,6]
    theta=ncon(tensors,connect,order)
    
    # SVD theta 
    theta=np.reshape(theta, [D*p,D*p])
    U,S,Vd=np.linalg.svd(theta)

    # Truncate the shared dimension of U,S,Vd back to D
    Gamma[A]=U[:,0:D]
    Lambda[A]=S[0:D]/np.trace(np.diag(S[0:D]))
    Gamma[B]=Vd[0:D,:]

    # Reshape back the Gamma tensor to original shape
    Gamma[A]=np.reshape(Gamma[A], [D,p,D])
    Gamma[A]=np.transpose(Gamma[A],[0,2,1])
    Gamma[B]=np.reshape(Gamma[B], [D,D,p])
    
    # Removing lambdas from updated Gammas
    Gamma[A]=ncon([np.diag(1./Lambda[B]),Gamma[A]],[[-1,1],[1,-2,-3]],[1])
    Gamma[B]=ncon([Gamma[B],np.diag(1./Lambda[B])],[[-1,1,-3],[1,-2]],[1])
    
# Gamma[A]=Gamma[A]/np.max(np.abs(Gamma[A]));  Gamma[B]=Gamma[B]/np.max(np.abs(Gamma[B]));
     

### Expectation Values
<img src="images/expectval.png" width="600"> 

In [13]:
#--- Calculating Bond Energies ---
E=[]
for i in range(len(Gamma)):
    if i%2==0:
        A=0; B=1
    else:
        A=1; B=0
        
    tensors=[np.diag(Lambda[B]),Gamma[A],np.diag(Lambda[A]),Gamma[B],np.diag(Lambda[B])]
    connect=[[-1,1],[1,2,-2],[2,3],[3,4,-4],[4,-3]]
    order=[1,4,3,2]
    theta=ncon(tensors,connect,order)
    
    Eb=ncon([theta,H_ITF,np.conj(theta)],[[1,2,3,4],[2,4,5,6],[1,5,3,6]],[5,6,1,2,3,4])
    E.append(Eb)
E_total=np.sum(E)
E0=E_total/2.0

print("E_total=", E_total)
print("E0=", E0)

f = lambda k,h : -2*np.sqrt(1+h**2-2*h*np.cos(k))/np.pi/2.
E0_exact = integrate.quad(f, 0, np.pi, args=(h,))[0]
print("E_exact =", E0_exact)

E_total= -2.009563588991422
E0= -1.004781794495711
E_exact = -1.0100252539845789
